In [1]:
from pathlib import Path
from skimage import filters
from skimage import io
from skimage import color
from skimage import feature
from skimage import morphology
from skimage import transform
from skimage import draw
from skimage import util
from skimage import measure
from skimage import exposure

import cv2
import re
import numpy as np


from collections import Counter
%matplotlib qt

### # V1 - AKTUALNE

In [91]:
# rect_points - (start_point, end_point), where p0 is top-left corner, p1 is down-right corner
def euclidean_distance(p1, p2):
        return pow(pow(p1[0]-p2[0],2)+pow(p1[1]-p2[1],2),0.5)


def sort_regions_by_area(regions, descending=True):     
    def func(region):
        return region.area

    regions = sorted(regions, key=func, reverse=descending)
    return regions

def get_digits_regions(image_digits):
    # Tutaj region to już chodzi o podobne jasności pikseli w sąsiedztwie.
    label_image = measure.label(image_digits)
    regions = measure.regionprops(label_image)
    print(len(regions))
    regions = sort_regions_by_area(regions, descending=True)
    regions = regions[:7] # bierzemy 7 bo tyle cyfr miał największy indeks

    #### SPRAWDZIC ###
#         while len(regions) < 6:
#             image_digits = morphology.erosion(image_digits, morphology.disk(1))
#             label_image = measure.label(image_digits)
#             regions = measure.regionprops(label_image)

    

    #######################################################################
#         region_mean_area = np.mean([reg.area for reg in regions])
#         region_std_area = np.std([reg.area for reg in regions])
#         regions = [reg for reg in regions if reg.area > region_mean_area/2]
    #######################################################################
#         region_mean_area = np.mean([reg.area for reg in regions])
#         print("region_mean_area: ", region_mean_area)
#         if len(regions) > 7:
#             region_mean_area = np.mean([reg.area for reg in regions])
#             regions = [reg for reg in regions if reg.area > region_mean_area]

#         counter = 1
#         while len(regions) < 6:
#             counter += 1
#             regions = [reg for reg in regions if reg.area > (region_mean_area-counter)]
#             if region_mean_area-counter < region_mean_area/2:
#                 break
#         regions = regions_above_mean.copy()    
    #######################################################################
    print("liczba wykrytych regionów (cyfr): ", len(regions))
    
    
    return regions

def get_list_of_rectangle_points(regions):
    rect_points = [get_two_points_to_create_rectangle_from_region(reg) for reg in regions]
    
    def func(points):
        p0, p1 = points
        return (p0[1] + p1[1])/2
    
    rect_points_sorted_by_distance_to_start_of_horizontal_axis = sorted(rect_points, key=func) 

    l1 = rect_points_sorted_by_distance_to_start_of_horizontal_axis[0:]
    l2 = rect_points_sorted_by_distance_to_start_of_horizontal_axis[1:]
    for reg1, reg2 in zip(l1, l2):
        half_diff = int((reg2[0][1] - reg1[1][1])/2)
        reg1[1][1] += half_diff
        reg2[0][1] -= half_diff
        
    return rect_points_sorted_by_distance_to_start_of_horizontal_axis
        

def get_two_points_to_create_rectangle_from_region(region):
    height_min = np.min(region.coords[:, 0])
    width_min = np.min(region.coords[:, 1])
    height_max = np.max(region.coords[:, 0])
    width_max = np.max(region.coords[:, 1])
    
    return [[height_min, width_min], [height_max, width_max]]

def scale_digit_image(image, scale=28):
    (h, w) = image.shape
    
    if abs(h-w) < 2:
        image = transform.resize(image, (scale,scale))
        return image
    if h > w: 
        half_diff = int((h-w)/2) # zakładamy, że jednak zawsze cyfra jest wyższa niż szersza
        
        # left border
        new_image_left_border = np.full((h,w+half_diff), fill_value=0)
        new_image_left_border[:,half_diff:] = image
        image = new_image_left_border
        
        
        # right border
        fill_value = (h-w) - 2*half_diff
        (h, w) = image.shape
        new_image_right_border = np.full((h,w+half_diff+fill_value), fill_value=0)
        new_image_right_border[:,:-half_diff-fill_value] = image
        image = new_image_right_border
        
    else:
        half_diff = int((w-h)/2) # zakładamy, że jednak zawsze cyfra jest wyższa niż szersza

        # bottom border  
        new_image_bot_border = np.full((h+half_diff,w), fill_value=0)
        new_image_bot_border[:-half_diff,:] = image
        image = new_image_bot_border

        # top border
        fill_value = (w-h) - 2*half_diff
        (h, w) = image.shape
        new_image_top_border = np.full((h+half_diff+fill_value,w), fill_value=0)
        new_image_top_border[half_diff+fill_value:,:] = image
        image = new_image_top_border
    
    image = util.img_as_ubyte(image)
    image = transform.resize(image, (scale,scale))
    image = util.img_as_bool(image)
    image = util.img_as_ubyte(image)
    
    return image


In [89]:
source_path = Path('../data/partial_results/wyciete_indeksy')

save_path = Path('../data/partial_results/wyciete_indeksy_wykryte_cyfry')
save_path.mkdir(parents=True, exist_ok=True)

images_directories = source_path.glob("*")

detections = []
for directory in images_directories:
    images_paths = directory.glob("*.png")
    
    # Utworzenie katalogu konkretnego zdjęcia.
    number_of_image = re.search('[0-9]+', directory.stem)[0]
    image_directory = save_path / number_of_image
    image_directory.mkdir(parents=True, exist_ok=True)
    
    for word_image_path in list(images_paths):
#         word_image_path = Path('../data/partial_results/wyciete_indeksy/1/2.png') # TODO - do usunięcia
        print(word_image_path) # TODO - do usunięcia
        
        # Utworzenie katalogu konkretnego indeksu.
        number_of_word = re.search('[0-9]+', word_image_path.stem)[0]
        word_directory = image_directory / str(number_of_word)
        word_directory.mkdir(parents=True, exist_ok=True)
    
        # Wczytanie obrazu
        word_image_org = io.imread(word_image_path)
        word_image = word_image_org.copy()
        word_image = color.rgb2gray(word_image)
        word_image = filters.gaussian(word_image)
        
        # Rozciąganie jasności obrazu.
        p2, p98 = np.percentile(word_image, (2, 98))
        word_image = exposure.rescale_intensity(word_image, in_range=(p2, p98))

        # Multio-Otsu dzieli obraz na 3 klasy o różnych jasnościach.
        thresholds = filters.threshold_multiotsu(word_image, classes=3)

        # Regions - to obraz, który ma wartości od 0 do 2. 
        # Wartość odpowiada regionowi, do którego należey dany piksel.
        otsu_regions = np.digitize(word_image, bins=thresholds)

        # Jeden z wykrytych regionów odpowiadał w większości jasności kratki, więc go usuwam.
        # Region trzeba traktować jako jakiś przedział wartości jasności w obrazie.
        image_removed_otsu_region = word_image*util.invert((otsu_regions == 1))
        
        # Po ponownym wykryciu regionów, jeden z nich pasował do cyfr więc użyłem go licząc, że są to cyfry.
        thresholds = filters.threshold_multiotsu(image_removed_otsu_region, classes=3)
        otsu_regions = np.digitize(word_image, bins=thresholds)
        image_digits = (otsu_regions==0)
        image_digits = morphology.dilation(image_digits, morphology.disk(1)) # DODANE

        regions = get_digits_regions(image_digits)
        detections.append(len(regions)) # TODO usunąć
        rect_points_sorted_by_distance_to_start_of_horizontal_axis = get_list_of_rectangle_points(regions)
        
        
               
        word_image = color.gray2rgb(word_image)  
        image_digits = util.img_as_ubyte(image_digits)
        temp_image = word_image_org.copy()
        for index_digit, (start_point, end_point) in enumerate(rect_points_sorted_by_distance_to_start_of_horizontal_axis):           
            rr, cc = draw.rectangle_perimeter(start_point, end_point, shape=word_image_org.shape)         
            temp_image[rr, cc] = (255,0+index_digit*30,0+index_digit*30)
            
            if start_point[1] > end_point[1]:
                break # TODO niepotrzebne?
                
#             for coord in region.coords:
#                 height = coord[0]
#                 width = coord[1]
#                 temp_image[(height,width)] = (255,0,0)
            
            
            one_digit =  image_digits[:, start_point[1]:end_point[1]+1]
            one_digit = scale_digit_image(one_digit, scale=28)
            io.imsave(arr=one_digit, fname=word_directory / '{}.png'.format(index_digit))
            
        io.imsave(arr=temp_image, fname=word_directory / 'index.png')
#         io.imshow(temp_image)
#         break
#     break

stats = Counter(detections)  
all_cases = sum(stats.values())
print("{}/{} == {}".format(stats[6], all_cases , (stats[6]/all_cases)))
print(stats)

..\data\partial_results\wyciete_indeksy\1\0.png
7
liczba wykrytych regionów (cyfr):  6
1:  Counter({0: 589, 255: 251})
2:  Counter({0: 1349, 255: 251})
3:  float64
4:  Counter({0: 659, 255: 125})
1:  Counter({0: 401, 255: 119})
2:  Counter({0: 1481, 255: 119})
3:  float64
4:  Counter({0: 724, 255: 60})
..\data\partial_results\wyciete_indeksy\1\1.png
18
liczba wykrytych regionów (cyfr):  6
1:  Counter({0: 1342, 255: 539})
2:  Counter({0: 2710, 255: 539})
3:  float64
4:  Counter({0: 651, 255: 133})
1:  Counter({0: 1930, 255: 578})
2:  Counter({0: 2671, 255: 578})
3:  float64
4:  Counter({0: 640, 255: 144})
1:  Counter({0: 2891, 255: 472})
2:  Counter({0: 3009, 255: 472})
3:  float64
4:  Counter({0: 674, 255: 110})
1:  Counter({0: 5428, 255: 1469})
2:  Counter({0: 13172, 255: 1469})
3:  float64
4:  Counter({0: 706, 255: 78})
1:  Counter({0: 1693, 255: 416})
2:  Counter({0: 2833, 255: 416})
3:  float64
4:  Counter({0: 685, 255: 99})
1:  Counter({0: 1641, 255: 468})
2:  Counter({0: 2781, 25

<ipython-input-89-44e1435c3a4b>:77: UserWarning: ..\data\partial_results\wyciete_indeksy_wykryte_cyfry\1\7\2.png is a low contrast image
  io.imsave(arr=one_digit, fname=word_directory / '{}.png'.format(index_digit))
<ipython-input-89-44e1435c3a4b>:77: UserWarning: ..\data\partial_results\wyciete_indeksy_wykryte_cyfry\1\9\0.png is a low contrast image
  io.imsave(arr=one_digit, fname=word_directory / '{}.png'.format(index_digit))


 Counter({0: 2032, 255: 438})
2:  Counter({0: 3787, 255: 438})
3:  float64
4:  Counter({0: 701, 255: 83})
..\data\partial_results\wyciete_indeksy\1\9.png
10
liczba wykrytych regionów (cyfr):  6
1:  Counter({0: 1074, 255: 32})
2:  Counter({0: 6209, 255: 32})
3:  float64
4:  Counter({0: 780, 255: 4})
1:  Counter({0: 2450, 255: 473})
2:  Counter({0: 5768, 255: 473})
3:  float64
4:  Counter({0: 722, 255: 62})
1:  Counter({0: 1637, 255: 575})
2:  Counter({0: 5666, 255: 575})
3:  float64
4:  Counter({0: 716, 255: 68})
1:  Counter({0: 1656, 255: 398})
2:  Counter({0: 5843, 255: 398})
3:  float64
4:  Counter({0: 735, 255: 49})
1:  Counter({0: 2109, 255: 419})
2:  Counter({0: 5822, 255: 419})
3:  float64
4:  Counter({0: 732, 255: 52})
1:  Counter({0: 2412, 255: 353})
2:  Counter({0: 5888, 255: 353})
3:  float64
4:  Counter({0: 740, 255: 44})
..\data\partial_results\wyciete_indeksy\10\0.png
2
liczba wykrytych regionów (cyfr):  2
..\data\partial_results\wyciete_indeksy\10\1.png
21
liczba wykrytyc

<ipython-input-89-44e1435c3a4b>:77: UserWarning: ..\data\partial_results\wyciete_indeksy_wykryte_cyfry\10\6\4.png is a low contrast image
  io.imsave(arr=one_digit, fname=word_directory / '{}.png'.format(index_digit))


1:  Counter({0: 1148, 255: 177})
2:  Counter({0: 2632, 255: 177})
3:  float64
4:  Counter({0: 735, 255: 49})
1:  Counter({0: 1811, 255: 415})
2:  Counter({0: 2394, 255: 415})
3:  float64
4:  Counter({0: 671, 255: 113})
1:  Counter({0: 1312, 255: 278})
2:  Counter({0: 2531, 255: 278})
3:  float64
4:  Counter({0: 711, 255: 73})
1:  Counter({0: 891, 255: 222})
2:  Counter({0: 2587, 255: 222})
3:  float64
4:  Counter({0: 723, 255: 61})
..\data\partial_results\wyciete_indeksy\10\9.png
14
liczba wykrytych regionów (cyfr):  6
1:  Counter({0: 1318, 255: 275})
2:  Counter({0: 3206, 255: 275})
3:  float64
4:  Counter({0: 724, 255: 60})
1:  Counter({0: 985, 255: 195})
2:  Counter({0: 3286, 255: 195})
3:  float64
4:  Counter({0: 739, 255: 45})
1:  Counter({0: 1536, 255: 470})
2:  Counter({0: 3011, 255: 470})
3:  float64
4:  Counter({0: 681, 255: 103})
1:  Counter({0: 1660, 255: 405})
2:  Counter({0: 3076, 255: 405})
3:  float64
4:  Counter({0: 695, 255: 89})
1:  Counter({0: 1170, 255: 187})
2:  Co

...

10
liczba wykrytych regionów (cyfr):  6
1:  Counter({0: 644, 255: 206})
2:  Counter({0: 2294, 255: 206})
3:  float64
4:  Counter({0: 726, 255: 58})
1:  Counter({0: 844, 255: 206})
2:  Counter({0: 2294, 255: 206})
3:  float64
4:  Counter({0: 723, 255: 61})
1:  Counter({0: 887, 255: 213})
2:  Counter({0: 2287, 255: 213})
3:  float64
4:  Counter({0: 722, 255: 62})
1:  Counter({0: 530, 255: 120})
2:  Counter({0: 2380, 255: 120})
3:  float64
4:  Counter({0: 747, 255: 37})
1:  Counter({0: 388, 255: 112})
2:  Counter({0: 2388, 255: 112})
3:  float64
4:  Counter({0: 744, 255: 40})
1:  Counter({0: 820, 255: 230})
2:  Counter({0: 2270, 255: 230})
3:  float64
4:  Counter({0: 716, 255: 68})
..\data\partial_results\wyciete_indeksy\7\11.png
37
liczba wykrytych regionów (cyfr):  6
1:  Counter({0: 5526, 255: 846})
2:  Counter({0: 10818, 255: 846})
3:  float64
4:  Counter({0: 728, 255: 56})
1:  Counter({0: 4508, 255: 784})
2:  Counter({0: 10880, 255: 784})
3:  float64
4:  Counter({0: 736, 255: 48})
1: 

<ipython-input-89-44e1435c3a4b>:77: UserWarning: ..\data\partial_results\wyciete_indeksy_wykryte_cyfry\7\9\0.png is a low contrast image
  io.imsave(arr=one_digit, fname=word_directory / '{}.png'.format(index_digit))
<ipython-input-89-44e1435c3a4b>:77: UserWarning: ..\data\partial_results\wyciete_indeksy_wykryte_cyfry\7\9\1.png is a low contrast image
  io.imsave(arr=one_digit, fname=word_directory / '{}.png'.format(index_digit))
<ipython-input-89-44e1435c3a4b>:77: UserWarning: ..\data\partial_results\wyciete_indeksy_wykryte_cyfry\7\9\3.png is a low contrast image
  io.imsave(arr=one_digit, fname=word_directory / '{}.png'.format(index_digit))
<ipython-input-89-44e1435c3a4b>:77: UserWarning: ..\data\partial_results\wyciete_indeksy_wykryte_cyfry\7\9\4.png is a low contrast image
  io.imsave(arr=one_digit, fname=word_directory / '{}.png'.format(index_digit))
<ipython-input-89-44e1435c3a4b>:77: UserWarning: ..\data\partial_results\wyciete_indeksy_wykryte_cyfry\7\9\5.png is a low contrast i

..\data\partial_results\wyciete_indeksy\8\0.png
3
liczba wykrytych regionów (cyfr):  3
1:  Counter({0: 408, 255: 152})
2:  Counter({0: 1073, 255: 152})
3:  float64
4:  Counter({0: 686, 255: 98})
1:  Counter({0: 322, 255: 62})
2:  Counter({0: 514, 255: 62})
3:  float64
4:  Counter({0: 699, 255: 85})
1:  Counter({0: 227, 255: 61})
2:  Counter({0: 263, 255: 61})
3:  float64
4:  Counter({0: 638, 255: 146})
..\data\partial_results\wyciete_indeksy\8\1.png
5
liczba wykrytych regionów (cyfr):  5
1:  Counter({0: 1575, 255: 705})
2:  Counter({0: 2895, 255: 705})
3:  float64
4:  Counter({0: 625, 255: 159})
1:  Counter({0: 2132, 255: 808})
2:  Counter({0: 2792, 255: 808})
3:  float64
4:  Counter({0: 605, 255: 179})
1:  Counter({0: 1668, 255: 552})
2:  Counter({0: 3048, 255: 552})
3:  float64
4:  Counter({0: 664, 255: 120})
1:  Counter({0: 1397, 255: 463})
2:  Counter({0: 3137, 255: 463})
3:  float64
4:  Counter({0: 685, 255: 99})
1:  Counter({0: 1792, 255: 488})
2:  Counter({0: 3112, 255: 488})
3:

3:  float64
4:  Counter({0: 650, 255: 134})
..\data\partial_results\wyciete_indeksy\8\8.png
7
liczba wykrytych regionów (cyfr):  6
1:  Counter({0: 597, 255: 235})
2:  Counter({0: 2469, 255: 235})
3:  float64
4:  Counter({0: 716, 255: 68})
1:  Counter({0: 1352, 255: 260})
2:  Counter({0: 2444, 255: 260})
3:  float64
4:  Counter({0: 713, 255: 71})
1:  Counter({0: 1242, 255: 422})
2:  Counter({0: 2282, 255: 422})
3:  float64
4:  Counter({0: 664, 255: 120})
1:  Counter({0: 1490, 255: 330})
2:  Counter({0: 2374, 255: 330})
3:  float64
4:  Counter({0: 692, 255: 92})
1:  Counter({0: 1170, 255: 442})
2:  Counter({0: 2262, 255: 442})
3:  float64
4:  Counter({0: 658, 255: 126})
1:  Counter({0: 1545, 255: 431})
2:  Counter({0: 2273, 255: 431})
3:  float64
4:  Counter({0: 661, 255: 123})
..\data\partial_results\wyciete_indeksy\8\9.png
10
liczba wykrytych regionów (cyfr):  6
1:  Counter({0: 800, 255: 169})
2:  Counter({0: 3080, 255: 169})
3:  float64
4:  Counter({0: 741, 255: 43})
1:  Counter({0: 1

3:  float64
4:  Counter({0: 730, 255: 54})
364/444 == 0.8198198198198198
Counter({6: 364, 1: 30, 3: 17, 2: 16, 4: 10, 5: 7})


Jakie problemy? przykład 3_5 - obraz 3, indeks 5
- dzieli liczby jak 2 czy 3 w połowie na dwie części góra i dół - DONE prawie - jeżeli podzieli liczbę na pół i te dwie połowy wykryje jako osobne regiony to mamy dwa razy 2 (bo wycinamy całą wysokość obrazu zawsze) - przykład 3_5
- cyfry zbyt blisko siebie, aż się prawie dotykają np. zdjęcie 1, ostatni wiersz cyfry 78
- czasami jasność kratki i cyfr jest tak bliska siebie, że cyfry łączą się poprzez kratkę i wykrywa dwie cyfry jako jeden region, np. 1_11 (82), 2_8 (23)
- trzeba coś wymyślić, zby wyeliminowac wykrywanie zbyt małych regionów (które nie są liczbami)
- wycinanie indeksów - może przydałoby się dodać lekki margines na około
- DONE jak jest bardzo cienka cyfra, np. jedynka w kształcie pojedynczej kreski to po przeskalowaniu wychodzi coś dziwnego, przykład 9_5
- 10_7 - dziwny przypadek, jakoś wymyślić żeby regiony mogły być tylko obok siebie
- przy wycinaniu cyfr też dodać lekki margines
- potrafi 4 podzielić na 2 części lewo-prawo 16_0
- no problem jak 7 cyfr 24_7
- może coś zrobić, aby regiony stykały się między sobą, region  jednej cyfry styka się z regionem drugiej cyfry

In [3]:
# zmienione v2
import cv2
source_path = Path('../data/partial_results/wyciete_indeksy')

save_path = Path('../data/partial_results/wyciete_indeksy_zmienione_v2')
save_path.mkdir(parents=True, exist_ok=True)

images_directories = source_path.glob("*")
for directory in images_directories:
    images_paths = directory.glob("*.png")
    number_of_image = re.search('[0-9]+', directory.stem)[0]
    save_directory = save_path / number_of_image
    save_directory.mkdir(parents=True, exist_ok=True)
    
    for image_path in list(images_paths):     
#         number_of_word = re.search('[0-9]+', image_path.stem)[0]
        print(image_path)
        img = io.imread(image_path)
        img = color.rgb2gray(img)
        
        # img = filters.gaussian(img)
        # Contrast stretching
        p2, p98 = np.percentile(img, (2, 98))
        img = exposure.rescale_intensity(img, in_range=(p2, p98))

        # thresh = filters.threshold_otsu(img, 3)
        # img = img > thresh
        # img = morphology.opening(img, morphology.disk(1))


        img = util.img_as_ubyte(img)
        img = cv2.Canny(image=img, threshold1=800, threshold2=1000)

        img = morphology.closing(img, morphology.disk(2))


        label_img = measure.label(img)
        regions = measure.regionprops(label_img)

        def func(region):
            return region.area

        img = color.gray2rgb(img)
        regions = sorted(regions, key=func, reverse=True)
        print(len(regions))
        for index_digit, region in enumerate(regions[:6]):
            height_min = np.min(region.coords[:, 0])
            width_min = np.min(region.coords[:, 1])
            height_max = np.max(region.coords[:, 0])
            width_max = np.max(region.coords[:, 1])
            rr, cc = draw.rectangle_perimeter((height_min, width_min), (height_max, width_max), shape=img.shape)         
            img[rr, cc] = (255,0,0)
        
        io.imsave(arr=img, fname=save_directory / "{}.png".format(number_of_word) )
        

# io.imshow(img)


..\data\partial_results\wyciete_indeksy\1\0.png
9
..\data\partial_results\wyciete_indeksy\1\1.png
19
..\data\partial_results\wyciete_indeksy\1\10.png
6
..\data\partial_results\wyciete_indeksy\1\11.png
18
..\data\partial_results\wyciete_indeksy\1\12.png
7
..\data\partial_results\wyciete_indeksy\1\2.png
7
..\data\partial_results\wyciete_indeksy\1\3.png
6
..\data\partial_results\wyciete_indeksy\1\4.png
8
..\data\partial_results\wyciete_indeksy\1\5.png
9
..\data\partial_results\wyciete_indeksy\1\6.png
5
..\data\partial_results\wyciete_indeksy\1\7.png
6
..\data\partial_results\wyciete_indeksy\1\8.png
7
..\data\partial_results\wyciete_indeksy\1\9.png
5
..\data\partial_results\wyciete_indeksy\10\0.png
5
..\data\partial_results\wyciete_indeksy\10\1.png
14
..\data\partial_results\wyciete_indeksy\10\10.png
6
..\data\partial_results\wyciete_indeksy\10\2.png
8
..\data\partial_results\wyciete_indeksy\10\3.png
8
..\data\partial_results\wyciete_indeksy\10\4.png
8
..\data\partial_results\wyciete_indek

..\data\partial_results\wyciete_indeksy\19\15.png
6
..\data\partial_results\wyciete_indeksy\19\16.png
9
..\data\partial_results\wyciete_indeksy\19\17.png
4
..\data\partial_results\wyciete_indeksy\19\18.png
3
..\data\partial_results\wyciete_indeksy\19\19.png
4
..\data\partial_results\wyciete_indeksy\19\2.png
6
..\data\partial_results\wyciete_indeksy\19\20.png
8
..\data\partial_results\wyciete_indeksy\19\21.png
8
..\data\partial_results\wyciete_indeksy\19\22.png
6
..\data\partial_results\wyciete_indeksy\19\3.png
3
..\data\partial_results\wyciete_indeksy\19\4.png
8
..\data\partial_results\wyciete_indeksy\19\5.png
7
..\data\partial_results\wyciete_indeksy\19\6.png
9
..\data\partial_results\wyciete_indeksy\19\7.png
8
..\data\partial_results\wyciete_indeksy\19\8.png
6
..\data\partial_results\wyciete_indeksy\19\9.png
9
..\data\partial_results\wyciete_indeksy\2\0.png
15
..\data\partial_results\wyciete_indeksy\2\1.png
13
..\data\partial_results\wyciete_indeksy\2\10.png
7
..\data\partial_results

1
..\data\partial_results\wyciete_indeksy\28\13.png
1
..\data\partial_results\wyciete_indeksy\28\14.png
1
..\data\partial_results\wyciete_indeksy\28\15.png
1
..\data\partial_results\wyciete_indeksy\28\2.png
1
..\data\partial_results\wyciete_indeksy\28\3.png
1
..\data\partial_results\wyciete_indeksy\28\4.png
8
..\data\partial_results\wyciete_indeksy\28\5.png
2
..\data\partial_results\wyciete_indeksy\28\6.png
4
..\data\partial_results\wyciete_indeksy\28\7.png
2
..\data\partial_results\wyciete_indeksy\28\8.png
1
..\data\partial_results\wyciete_indeksy\28\9.png
1
..\data\partial_results\wyciete_indeksy\3\0.png
3
..\data\partial_results\wyciete_indeksy\3\1.png
5
..\data\partial_results\wyciete_indeksy\3\10.png
5
..\data\partial_results\wyciete_indeksy\3\11.png
6
..\data\partial_results\wyciete_indeksy\3\12.png
10
..\data\partial_results\wyciete_indeksy\3\13.png
18
..\data\partial_results\wyciete_indeksy\3\14.png
6
..\data\partial_results\wyciete_indeksy\3\2.png
7
..\data\partial_results\wyc

In [54]:
# V3
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from skimage import data
from skimage.filters import threshold_multiotsu


source_path = Path('../data/partial_results/wyciete_indeksy')

save_path = Path('../data/partial_results/wyciete_indeksy_zmienione_v3')
save_path.mkdir(parents=True, exist_ok=True)

images_directories = source_path.glob("*")
cnt = 0
cnt_good = 0
detections = []
for directory in images_directories:
    images_paths = directory.glob("*.png")
    number_of_image = re.search('[0-9]+', directory.stem)[0]
    image_directory = save_path / number_of_image
    image_directory.mkdir(parents=True, exist_ok=True)
    
    for word_image_path in list(images_paths)[11:]:
        word_image_path = Path('../data/partial_results/wyciete_indeksy/3/3.png')
#         number_of_word = re.search('[0-9]+', word_image_path.stem)[0]
        cnt += 1
        
#         word_directory = image_directory / str(number_of_word)
#         word_directory.mkdir(parents=True, exist_ok=True)
    
        print(word_image_path)
        
        word_image_org = io.imread(word_image_path)
        word_image = word_image_org.copy()
        word_image = color.rgb2gray(word_image)
        

        word_image = filters.gaussian(word_image)
        # Contrast stretching
        p2, p98 = np.percentile(word_image, (2, 98))
        word_image = exposure.rescale_intensity(word_image, in_range=(p2, p98))
        

        # Applying multi-Otsu threshold for the default value, generating
        # three classes. 
        thresholds = threshold_multiotsu(word_image, classes=3)

        # Using the threshold values, we generate the three regions.
        regions = np.digitize(word_image, bins=thresholds)
        

        # Jeden z wykrytych regionów odpowiadał w większości jasności kratki, więc go usuwam.
        # Region trzeba traktować jako jakiś przedział wartości jasności w obrazie.
        img2 = word_image*util.invert((regions == 1))
        
        # Po ponownym wykryciu regionów, jeden z nich pasował do cyfr więc użyłem go licząc, że są to cyfry.
        thresholds = threshold_multiotsu(img2, classes=3)
        regions = np.digitize(word_image, bins=thresholds)
        img3 = (regions==0)
        
        img3 = util.img_as_ubyte(img3)
        img3 = morphology.dilation(img3, morphology.disk(1))
        io.imshow(img3)
        break
        io.imsave(arr=img3, fname=image_directory / "{}.png".format(number_of_word) )
    break

..\data\partial_results\wyciete_indeksy\3\3.png


In [13]:
# import cv2
# source_path = Path('../data/partial_results/wyciete_indeksy')

# save_path = Path('../data/partial_results/wyciete_indeksy_zmienione')
# save_path.mkdir(parents=True, exist_ok=True)

# images_directories = source_path.glob("*")
# for directory in images_directories:
#     images_paths = directory.glob("*.png")
#     number_of_image = re.search('[0-9]+', directory.stem)[0]
#     save_directory = save_path / number_of_image
#     save_directory.mkdir(parents=True, exist_ok=True)
    
#     for image_path in list(images_paths):        
#         print(image_path)
#         img = io.imread(image_path)
#         img = color.rgb2gray(img)

#         img = filters.gaussian(img)
#         # Contrast stretching
#         p2, p98 = np.percentile(img, (2, 98))
#         img = exposure.rescale_intensity(img, in_range=(p2, p98))
        
#         img = util.img_as_ubyte(img)
#         img = cv2.Canny(image=img, threshold1=300, threshold2=400)
        
# #         img = morphology.dilation(img, morphology.disk(3))
#         io.imshow(img)
        
        
#         break
#     break

..\data\partial_results\wyciete_indeksy\1\0.png
